# MNIST Digits Classification using Transfer Learning with ResNet50 model



# Data Pre-Processing

## Imports


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import MNIST
from sklearn.metrics import classification_report

## Loading the MNIST Dataset






In [2]:
# Custom transform to convert MNIST to RGB and resize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 12.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 342kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.17MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.97MB/s]

Training samples: 60000
Test samples: 10000


# Model Architecture

## Defining the model

In [3]:
class MNISTResNet50(nn.Module):
    def __init__(self, num_classes=10, freeze_base=True):
        super(MNISTResNet50, self).__init__()

        # Load pre-trained ResNet50
        self.resnet = models.resnet50(pretrained=True)

        # Remove the final classification layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        # Freeze base model if specified
        if freeze_base:
            for param in self.resnet.parameters():
                param.requires_grad = False

        # Add custom classification head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 128),  # ResNet50 final feature size is 2048
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        features = self.resnet(x)
        output = self.classifier(features)
        return output

    def unfreeze_base(self):
        """Unfreeze base ResNet50 layers for fine-tuning"""
        for param in self.resnet.parameters():
            param.requires_grad = True

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MNISTResNet50(freeze_base=True).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Device: {device}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 103MB/s]


Device: cuda
Total parameters: 23,771,594
Trainable parameters: 263,562


## Training & Evaluation functions

In [5]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        if batch_idx % 500 == 0:
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    test_loss /= len(test_loader)
    test_acc = 100. * correct / total
    return test_loss, test_acc


# Model Training

## Initial Training (Frozen Base)

In [6]:
# Setup for initial training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Starting initial training with frozen base layers...")
print("="*50)

# Train for 5 epochs with frozen base
for epoch in range(5):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f'Epoch {epoch+1}/5:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print('-' * 30)

initial_accuracy = test_acc
print(f"Initial training completed. Test accuracy: {initial_accuracy:.2f}%")


Starting initial training with frozen base layers...
Batch 0, Loss: 2.3040
Batch 500, Loss: 0.2492
Batch 1000, Loss: 0.2247
Batch 1500, Loss: 0.1147
Epoch 1/5:
  Train Loss: 0.4101, Train Acc: 87.31%
  Test Loss: 0.1891, Test Acc: 94.14%
------------------------------
Batch 0, Loss: 0.4773
Batch 500, Loss: 0.1643
Batch 1000, Loss: 0.6642
Batch 1500, Loss: 0.1579
Epoch 2/5:
  Train Loss: 0.2288, Train Acc: 92.63%
  Test Loss: 0.2093, Test Acc: 93.06%
------------------------------
Batch 0, Loss: 0.2829
Batch 500, Loss: 0.4529
Batch 1000, Loss: 0.3190
Batch 1500, Loss: 0.1951
Epoch 3/5:
  Train Loss: 0.2053, Train Acc: 93.27%
  Test Loss: 0.1355, Test Acc: 95.72%
------------------------------
Batch 0, Loss: 0.3996
Batch 500, Loss: 0.0510
Batch 1000, Loss: 0.1436
Batch 1500, Loss: 0.1295
Epoch 4/5:
  Train Loss: 0.1806, Train Acc: 94.16%
  Test Loss: 0.1732, Test Acc: 94.43%
------------------------------
Batch 0, Loss: 0.1716
Batch 500, Loss: 0.0188
Batch 1000, Loss: 0.1068
Batch 1500, 

## Fine-tuning (Unfrozen Base)

In [ ]:
# Unfreeze base model for fine-tuning
model.unfreeze_base()

# Use lower learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Count trainable parameters after unfreezing
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Fine-tuning - Trainable parameters: {trainable_params:,}")

print("\nStarting fine-tuning with unfrozen base layers...")
print("="*50)

# Fine-tune for 3 epochs
for epoch in range(3):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f'Fine-tune Epoch {epoch+1}/3:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print('-' * 30)

final_accuracy = test_acc
print(f"Fine-tuning completed. Final test accuracy: {final_accuracy:.2f}%")


Fine-tuning - Trainable parameters: 23,771,594

Starting fine-tuning with unfrozen base layers...
Batch 0, Loss: 0.0682
Batch 500, Loss: 0.0118
Batch 1000, Loss: 0.0264
Batch 1500, Loss: 0.0528
Fine-tune Epoch 1/3:
  Train Loss: 0.0684, Train Acc: 98.05%
  Test Loss: 0.0236, Test Acc: 99.16%
------------------------------
Batch 0, Loss: 0.0064
Batch 500, Loss: 0.0043
Batch 1000, Loss: 0.0181
Batch 1500, Loss: 0.0037
Fine-tune Epoch 2/3:
  Train Loss: 0.0331, Train Acc: 98.98%
  Test Loss: 0.0278, Test Acc: 99.24%
------------------------------
Batch 0, Loss: 0.0093
Batch 500, Loss: 0.0009
Batch 1000, Loss: 0.0469
Batch 1500, Loss: 0.0005


# Model Evaluation

In [9]:
def detailed_evaluation(model, test_loader, device):
    model.eval()
    all_predicted = []
    all_targets = []
    sample_confidences = []

    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            probabilities = torch.softmax(output, dim=1)
            _, predicted = torch.max(output, 1)

            all_predicted.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

            # Collect sample confidences for first batch
            if i == 0:
                for j in range(min(10, len(probabilities))):
                    confidence = torch.max(probabilities[j]).item()
                    sample_confidences.append((predicted[j].item(), target[j].item(), confidence))

    return all_predicted, all_targets, sample_confidences

# Perform detailed evaluation
predicted, targets, sample_conf = detailed_evaluation(model, test_loader, device)

print("\nSample predictions vs actual:")
for i, (pred, actual, conf) in enumerate(sample_conf):
    print(f"Image {i}: Predicted={pred}, Actual={actual}, Confidence={conf:.3f}")

print(f"\nOverall Accuracy: {final_accuracy:.2f}%")
print(f"Improvement from fine-tuning: {final_accuracy - initial_accuracy:.2f}%")

# Classification report
print("\nClassification Report:")
print(classification_report(targets, predicted))


Sample predictions vs actual:
Image 0: Predicted=7, Actual=7, Confidence=1.000
Image 1: Predicted=2, Actual=2, Confidence=0.999
Image 2: Predicted=1, Actual=1, Confidence=1.000
Image 3: Predicted=0, Actual=0, Confidence=1.000
Image 4: Predicted=4, Actual=4, Confidence=1.000
Image 5: Predicted=1, Actual=1, Confidence=1.000
Image 6: Predicted=4, Actual=4, Confidence=0.996
Image 7: Predicted=9, Actual=9, Confidence=0.998
Image 8: Predicted=5, Actual=5, Confidence=1.000
Image 9: Predicted=9, Actual=9, Confidence=1.000

Overall Accuracy: 99.23%
Improvement from fine-tuning: 3.87%

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       1.00      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6  